In [1]:
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
import os
import json

In [2]:
# 设置OpenAI的API Key
os.environ["OPENAI_API_KEY"] = "sk-CXiYydndph1xjzs5dBR6qDWDRUMYXAYrkWlDhstCWajXXRvC"
os.environ["OPENAI_API_BASE"] = 'https://api.chatanywhere.tech/v1'

# 设置OpenAI的模型
base_embeddings = OpenAIEmbeddings()
llm = OpenAI(temperature=0)

In [3]:
# 设置hyde的prompt
prompt_template = """请回答用户关于中山大学信息的查询
查询: {query}
回答: """
prompt = PromptTemplate(input_variables=["query"], template=prompt_template)

In [4]:
vector_store = Chroma(embedding_function=base_embeddings, persist_directory='D:/codefile/sysu-introducer-plus/data/vectorstores')

In [21]:
# 加载 JSON 文件
with open("D:\\codefile\\sysu-introducer-plus\\data\\database.json", "r") as f:
    data = json.load(f)

# 创建文档列表
documents = [Document(page_content=prompt_template.format(query=value['query'])+value['document'], metadata={'id': key, 'keyword': value['metadata']}) for key, value in data.items()]

vector_store.add_documents(documents)

In [27]:
# 计数
add_doc_count = 0
for doc in documents:
    # print(doc.metadata['id'])
    if any(item['id'] == doc.metadata['id'] for item in vector_store.get()['metadatas']):
        continue
    print(doc.metadata['id'])
    vector_store.add_document(doc)
    add_doc_count += 1
return add_doc_count == 0

In [19]:
query = "中山大学简介"
llm_chain = LLMChain(llm=llm, prompt=prompt)
query = prompt_template.format(query=query)
res =  llm_chain.invoke(query)['text']
query += res
print(query)
docs = vector_store.similarity_search_with_score(query, k=3)
for doc in docs:
    print(doc[1])
    print(doc[0].page_content)
query += '\n' + res
# print(query)
docs = vector_store.similarity_search_with_score(query, k=3)
print("====================================")
for doc in docs:
    print(doc[1])
    print(doc[0].page_content)

d:\miniconda\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


请回答用户关于中山大学信息的查询
查询: 中山大学简介
回答: 中山大学是一所位于广东省广州市的综合性大学，是中国近代著名民主革命家孙中山先生创办的国内第一所民办高等学校，也是中国最早的现代国立大学之一。中山大学是一所历史悠久、享有盛誉的高等学府，被誉为“百年名校、国立典范”。学校现有三个校区，分别是广州校区、珠海校区和深圳校区，拥有多个学科门类，是一所具有国际影响力的综合性研究型大学。
0.10849601775407791
请回答用户关于中山大学信息的查询
查询: 中山大学成立的校史
回答: 1924年,孙中山亲手将广州地区多所高校整合创立国立广东大学。1926年定名为国立中山大学。如今该校由1952年院系调整后分设的中山大学和中山医科大学于2001年10月合并而成。
0.11983279883861542
请回答用户关于中山大学信息的查询
查询: 中山大学简介
回答: 中山大学(Sun Yat-sen University),简称“中大”,位于广东省,由中华人民共和国教育部直属,是教育部、国家国防科技工业局和广东省共建的综合性全国重点大学,位列国家“双一流”、“985工程”、“211工程”,入选国家“珠峰计划”、“111计划”、“2011计划”、卓越法律人才教育培养计划、卓越医生教育培养计划、国家大学生创新性实验计划、国家级大学生创新创业训练计划、国家建设高水平大学公派研究生项目、新工科研究与实践项目、全国深化创新创业教育改革示范高校、国家大学生文化素质教育基地、国家创新人才培养示范基地、国家国际科技合作基地、首批高等学校科技成果转化和技术转移基地、学位授权自主审核单位等,是环太平洋大学联盟、中国高校行星科学联盟、中国人工智能教育联席会、中国自由贸易试验区研究院联盟、大学通识教育联盟、粤港澳高校联盟、粤港澳大湾区物流与供应链创新联盟成员。 
0.13759306073188782
请回答用户关于中山大学信息的查询
查询: 中山大学的1928年历史
回答: 1928年1月,大学院将全国划分为若干大学区(即北平、江苏、浙江、广州),以所辖区域命名。每大学区设大学一所,除在广州者永远定名为中山大学以纪念中国国民党总理孙中山外,均以所在地命名
0.11829142272472382
请回答用户关于中山大学信息的查询
查询: 中山大学成立的校史
回答: 1924年,孙中

In [7]:
vector_store.get()

{'ids': ['007e86c3-fac0-4ffc-b269-7f7bace64ed2',
  '00897980-ead7-4ce4-9b89-a962904139ff',
  '00df42da-edc7-4404-9e29-4fa6c2de7ac9',
  '014c2a25-ccec-4b98-8a0c-f21143b32693',
  '0255aa70-6b82-4abd-8c15-eebcd1936a49',
  '029ab39f-00bd-4c38-bced-bcb2bc7faa3d',
  '033eabc0-fccd-4999-9d8a-a20b687eab9a',
  '040f17b4-1f76-40d0-b5f3-701d249deb97',
  '065b9e58-1cd4-4316-84b8-6512e07e26ff',
  '068c5fcf-b805-4fb7-bf87-61eae134736e',
  '06ae165f-9ed1-4dcf-b02a-cccbc2e9daa9',
  '06e25ceb-5dbe-41c6-83ea-a04a62e3c999',
  '07c81eb8-63dc-45a2-ba85-5ddc91ad5f24',
  '08e31536-3465-4cfc-9921-bc6cf524002d',
  '09325e52-8479-4006-808b-206a68c65d1f',
  '09387e62-0216-441c-8cb0-6ebb8e457939',
  '0b18e501-9fcb-478a-91f8-4d7e9fd12275',
  '0b787ffc-61c9-49f2-b487-0b7ec29a1846',
  '0c505752-f0a0-4ed9-894b-ffb05c1b9b46',
  '0d562314-f15c-4dae-872f-1afc59612e0d',
  '0d6bb15c-daed-48cc-8b04-11f7bba29f81',
  '0e2ca7ad-8d19-4348-9471-944966eff952',
  '0e55c475-52a6-4715-8b33-e562df7a343a',
  '0e5c43ff-7715-431a-bc45-

In [8]:
print(documents[0])

page_content='请回答用户关于中山大学信息的查询\n查询: 中山大学简介\n回答: 中山大学(Sun Yat-sen University),简称“中大”,位于广东省,由中华人民共和国教育部直属,是教育部、国家国防科技工业局和广东省共建的综合性全国重点大学,位列国家“双一流”、“985工程”、“211工程”,入选国家“珠峰计划”、“111计划”、“2011计划”、卓越法律人才教育培养计划、卓越医生教育培养计划、国家大学生创新性实验计划、国家级大学生创新创业训练计划、国家建设高水平大学公派研究生项目、新工科研究与实践项目、全国深化创新创业教育改革示范高校、国家大学生文化素质教育基地、国家创新人才培养示范基地、国家国际科技合作基地、首批高等学校科技成果转化和技术转移基地、学位授权自主审核单位等,是环太平洋大学联盟、中国高校行星科学联盟、中国人工智能教育联席会、中国自由贸易试验区研究院联盟、大学通识教育联盟、粤港澳高校联盟、粤港澳大湾区物流与供应链创新联盟成员。 ' metadata={'id': '0', 'keyword': '简介,地位'}


In [17]:
# print(vector_store.get()['metadatas'])
search_id = '11111'
id_exists = any(item['id'] == search_id for item in vector_store.get()['metadatas'])

In [11]:
vector_store.get()

{'ids': ['0064e422-6a9f-4b3a-82fd-e69488dc6e94',
  '006eb36d-dc84-4024-a863-3d6ead99a40f',
  '00f90164-4ada-471e-afd4-973900c3d739',
  '01016bf9-b34a-4d7f-b593-080e05be3557',
  '0137d1ff-21c6-4521-877e-630f75b55f85',
  '01457377-057e-40a8-b398-9991ceefc839',
  '0159761c-623a-4c97-ad81-09b2829c20ba',
  '015e8644-ab61-41de-aaeb-2701ea743bf8',
  '018459da-5756-4bd0-b256-8e08bf46d4e4',
  '019c793d-ef2e-4620-a6f2-2be61429064a',
  '01dee537-7e5c-4bfd-b279-7fa2a6804b29',
  '020ee417-ca1b-454e-95c8-a87b2625a752',
  '0216cd83-3c4f-44ba-b709-37fc94991c84',
  '022b218a-6650-47fe-8e14-9a09a8bd130a',
  '025abde1-2622-4a7c-836a-86cd9390f076',
  '02afe110-a6ca-4144-8eeb-c1b253b4ef13',
  '02e24f30-b252-4484-b8fa-1e76af6df2ea',
  '034a65af-684e-4642-acd5-1933b7bd25e8',
  '0390227e-d543-45e5-97a8-ddde6e97b3ad',
  '03d67afe-c5d7-4345-9d74-8e0bfe662f5a',
  '03efbdab-5f40-4208-99a7-16ced640b1b0',
  '043e5fb4-b933-4d08-83db-ec14fe26e67a',
  '047791f6-5212-4a6a-9c8c-f68f61306bad',
  '04938020-8225-4ac4-9967-

In [21]:
# 加载 JSON 文件
with open("D:\\codefile\\sysu-introducer-plus\\data\\database.json", "r") as f:
    data = json.load(f)

# 创建文档列表
documents = [Document(page_content=prompt_template.format(query=value['query'])+value['document'], metadata={'id': key, 'query': value['query'], 'document': value['document'], 'keyword': value['metadata']}) for key, value in data.items()]

print(len(documents))
aa = 0
for doc in documents:
    aa += 1
    # 如果文档已经存在，则从documents删去
    if any(item['id'] == doc.metadata['id'] for item in vector_store.get()['metadatas']):
        documents.remove(doc)
        
        continue
    print("aaaaaaaa")
    
print(aa)
# add_doc_count = len(documents)
# print(f'Building index for {add_doc_count} documents...')
# documents

342
342


In [20]:
print(type(documents))

<class 'list'>
